In [1]:
! pip install -q geopandas rasterio

     |████████████████████████████████| 972kB 4.0MB/s 
     |████████████████████████████████| 18.3MB 238kB/s 
     |████████████████████████████████| 6.5MB 40.9MB/s 
     |████████████████████████████████| 14.8MB 338kB/s 


In [2]:
import os
import glob
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import windows, transform
from rasterio.enums import Resampling
from rasterio.plot import show
from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
SHP_DIR = '/content/drive/Shareddrives/stand_mapping/data/interim'

WA11_SHP = 'washington_utm11n_training_quads_epsg6340.shp'
WA10_SHP = 'washington_utm10n_training_quads_epsg6339.shp'
OR10_SHP = 'oregon_utm10n_training_quads_epsg6339.shp'
OR11_SHP = 'oregon_utm11n_training_quads_epsg6340.shp'

or10_gdf = gpd.read_file(os.path.join(SHP_DIR, OR10_SHP))
or11_gdf = gpd.read_file(os.path.join(SHP_DIR, OR11_SHP))
wa10_gdf = gpd.read_file(os.path.join(SHP_DIR, WA10_SHP))
wa11_gdf = gpd.read_file(os.path.join(SHP_DIR, WA11_SHP))

# Get input rasters to local machine for faster I/O

In [6]:
IN_DIR_ROOT = '/content/drive/Shareddrives/stand_mapping/data/interim/training_tiles'
OUT_DIR_ROOT = '/content/drive/Shareddrives/stand_mapping/data/processed/training_tiles'

## Resample the rasters to 1m

In [7]:
def get_tile_bbox(geometry):
    """Rounds bounding box coordinates outward to nearest integer"""
    xmin, ymin, xmax, ymax = geometry.bounds
    xmin, ymin = np.floor((xmin, ymin)).astype(int)
    xmax, ymax = np.ceil((xmax, ymax)).astype(int)
    bbox = (xmin, ymin, xmax, ymax)
    return bbox
    
def resample_landsat(tile_id, state, year, bbox):
    leafon_name = f'{tile_id}_landsat-leaf-on_{year}.tif'
    leafoff_name = f'{tile_id}_landsat-leaf-off_{year}.tif'
    leafon_in = os.path.join(IN_DIR_ROOT, state, 'landsat', str(year), leafon_name)
    leafoff_in = os.path.join(IN_DIR_ROOT, state, 'landsat', str(year), leafoff_name)
    leafon_out = os.path.join(OUT_DIR_ROOT, state, 'landsat', str(year), leafon_name)
    leafoff_out = os.path.join(OUT_DIR_ROOT, state, 'landsat', str(year), leafoff_name)

    xmin, ymin, xmax, ymax = bbox
    width = xmax-xmin
    height = ymax-ymin
    out_trf = transform.from_bounds(xmin, ymin, xmax, ymax, width=width, height=height)

    inpath = leafon_in
    outpath = leafon_out
    if not os.path.exists(outpath) and os.path.exists(inpath):
        with rasterio.open(inpath) as src:
            window = windows.from_bounds(xmin, ymin, xmax, ymax, transform=src.transform)
            out_prof = src.profile.copy()
            out_prof.update(width=width, height=height, transform=out_trf)
            data = src.read(window=window, out_shape=(src.count, height, width), resampling=Resampling.nearest)
        with rasterio.open(outpath, mode='w', **out_prof) as dst:
            dst.write(data)
    
    inpath = leafoff_in
    outpath = leafoff_out
    if not os.path.exists(outpath) and os.path.exists(inpath):
        with rasterio.open(inpath) as src:
            window = windows.from_bounds(xmin, ymin, xmax, ymax, transform=src.transform)
            out_prof = src.profile.copy()
            out_prof.update(width=width, height=height, transform=out_trf)
            data = src.read(window=window, out_shape=(src.count, height, width), resampling=Resampling.nearest)
        with rasterio.open(outpath, mode='w', **out_prof) as dst:
            dst.write(data)

    return leafon_out, leafoff_out

In [ ]:
GDFS = [wa10_gdf, wa11_gdf]
YEARS = [2009, 2011, 2015, 2017]
STATE = 'washington'

for year in YEARS:
    with tqdm(total=len(GDFS[0])+len(GDFS[1]), desc=str(year)) as pbar:
        for gdf in GDFS:
            for idx, row in gdf.iterrows():
                bbox = get_tile_bbox(row.geometry)
                cell_id = row.CELL_ID
                resample_landsat(cell_id, STATE, year, bbox)
                pbar.update()

In [12]:
GDFS = [or10_gdf, or11_gdf]
YEARS = [2009, 2011, 2014, 2016]
STATE = 'oregon'

for year in YEARS:
    with tqdm(total=len(GDFS[0])+len(GDFS[1]), desc=str(year)) as pbar:
        for gdf in GDFS:
            for idx, row in gdf.iterrows():
                bbox = get_tile_bbox(row.geometry)
                cell_id = row.CELL_ID
                resample_landsat(cell_id, STATE, year, bbox)
                pbar.update()

In [ ]:
def resample_landtrendr(tile_id, state, year, bbox):
    fname = f'{tile_id}_landtrendr_{year}.tif'
    inpath = os.path.join(IN_DIR_ROOT, state, 'landtrendr', str(year), fname)
    outpath = os.path.join(OUT_DIR_ROOT, state, 'landtrendr', str(year), fname)

    xmin, ymin, xmax, ymax = bbox
    width = xmax-xmin
    height = ymax-ymin
    out_trf = transform.from_bounds(xmin, ymin, xmax, ymax, width=width, height=height)

    if not os.path.exists(outpath) and os.path.exists(inpath):
        with rasterio.open(inpath) as src:
            window = windows.from_bounds(xmin, ymin, xmax, ymax, transform=src.transform)
            out_prof = src.profile.copy()
            out_prof.update(width=width, height=height, transform=out_trf)
            data = src.read(window=window, out_shape=(src.count, height, width), resampling=Resampling.nearest)
        with rasterio.open(outpath, mode='w', **out_prof) as dst:
            dst.write(data)
    
    return outpath

In [ ]:
YEARS = [2009, 2011, 2014, 2016]
STATE = 'oregon'
for year in YEARS:
  os.makedirs(os.path.join(OUT_DIR_ROOT, STATE, 'landtrendr', str(year)), exist_ok=True)

YEARS = [2009, 2011, 2015, 2017]
STATE = 'washington'
for year in YEARS:
  os.makedirs(os.path.join(OUT_DIR_ROOT, STATE, 'landtrendr', str(year)), exist_ok=True)

In [ ]:
GDFS = [or10_gdf, or11_gdf]
YEARS = [2009, 2011, 2014, 2016]
STATE = 'oregon'

for year in YEARS:
    with tqdm(total=len(GDFS[0])+len(GDFS[1]), desc=str(year)) as pbar:
        for gdf in GDFS:
            for idx, row in gdf.iterrows():
                bbox = get_tile_bbox(row.geometry)
                cell_id = row.CELL_ID
                resample_landtrendr(cell_id, STATE, year, bbox)
                pbar.update()

In [ ]:
GDFS = [wa10_gdf, wa11_gdf]
YEARS = [2009, 2011, 2015, 2017]
STATE = 'washington'

for year in YEARS:
    with tqdm(total=len(GDFS[0])+len(GDFS[1]), desc=str(year)) as pbar:
        for gdf in GDFS:
            for idx, row in gdf.iterrows():
                bbox = get_tile_bbox(row.geometry)
                cell_id = row.CELL_ID
                resample_landtrendr(cell_id, STATE, year, bbox)
                pbar.update()